In [26]:
import csv
import os 
import numpy as np

In [2]:
num_coords =501
landMarkes =['class']
for val in range(1, num_coords+1):
    landMarkes +=['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [3]:
with open('D:/oprncv/test_vf/test.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landMarkes)    
    
    

In [4]:
!pip install mediapipe opencv-python pandas scikit-learn

In [24]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [25]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [5]:
class_name = "unfocus"

In [6]:
cap = cv2.VideoCapture('D:/oprncv/UNFOCUS.mp4')
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('D:/oprncv/test_vf/test.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [23]:
#training model
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('D:/oprncv/test_vf/test.csv')

In [6]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,focus,0.501528,0.474411,-0.082952,0.995225,0.496069,0.452557,-0.067683,0.994081,0.495498,...,0.002650,0.0,0.507732,0.469205,0.015705,0.0,0.507850,0.467126,0.016655,0.0
1,focus,0.500289,0.476802,-0.116772,0.994114,0.495189,0.455198,-0.101474,0.992979,0.494785,...,0.002386,0.0,0.504121,0.467182,0.014150,0.0,0.504187,0.465272,0.015016,0.0
2,focus,0.499857,0.478029,-0.119379,0.992658,0.494812,0.456313,-0.104074,0.991732,0.494425,...,0.001432,0.0,0.501672,0.466993,0.012090,0.0,0.501839,0.465008,0.012898,0.0
3,focus,0.499697,0.479158,-0.107869,0.991428,0.494598,0.457348,-0.089532,0.990745,0.494180,...,0.003161,0.0,0.506961,0.469361,0.015826,0.0,0.507163,0.467385,0.016747,0.0
4,focus,0.499500,0.480281,-0.066502,0.991234,0.494484,0.458383,-0.047151,0.990624,0.494068,...,0.002276,0.0,0.503024,0.467659,0.013195,0.0,0.503178,0.465755,0.013985,0.0


In [7]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [22]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [11]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [12]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

D:\programs\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [14]:
fit_models['rc'].predict(X_test)

array(['unfocus', 'unfocus', 'unfocus', 'focus', 'focus', 'unfocus',
       'unfocus', 'unfocus', 'focus', 'unfocus', 'unfocus', 'unfocus',
       'unfocus', 'unfocus', 'unfocus', 'unfocus', 'focus', 'unfocus',
       'focus', 'focus', 'unfocus', 'focus', 'unfocus', 'unfocus',
       'unfocus', 'focus', 'unfocus', 'unfocus', 'unfocus', 'unfocus',
       'focus', 'unfocus', 'focus', 'focus', 'focus', 'unfocus',
       'unfocus', 'focus', 'unfocus', 'unfocus', 'unfocus', 'unfocus',
       'focus', 'unfocus', 'unfocus', 'unfocus', 'unfocus', 'unfocus',
       'unfocus', 'unfocus', 'unfocus', 'focus', 'focus', 'unfocus',
       'unfocus', 'unfocus', 'unfocus', 'unfocus', 'unfocus', 'unfocus',
       'unfocus', 'unfocus', 'unfocus', 'unfocus', 'focus', 'unfocus',
       'unfocus', 'unfocus', 'unfocus', 'unfocus', 'focus', 'focus',
       'unfocus', 'unfocus', 'unfocus', 'unfocus', 'unfocus', 'unfocus',
       'unfocus', 'unfocus', 'unfocus', 'unfocus', 'unfocus', 'focus',
       'unfocus', 

In [21]:
#vEvaluate model
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 


In [16]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 0.9958419958419958
gb 0.997920997920998


In [17]:
fit_models['rf'].predict(X_test)

array(['unfocus', 'unfocus', 'unfocus', 'focus', 'focus', 'unfocus',
       'unfocus', 'unfocus', 'focus', 'unfocus', 'unfocus', 'unfocus',
       'unfocus', 'unfocus', 'unfocus', 'unfocus', 'focus', 'unfocus',
       'unfocus', 'focus', 'unfocus', 'focus', 'unfocus', 'unfocus',
       'unfocus', 'focus', 'unfocus', 'unfocus', 'unfocus', 'unfocus',
       'focus', 'focus', 'focus', 'focus', 'focus', 'unfocus', 'unfocus',
       'focus', 'unfocus', 'unfocus', 'unfocus', 'unfocus', 'focus',
       'unfocus', 'unfocus', 'unfocus', 'unfocus', 'unfocus', 'unfocus',
       'unfocus', 'unfocus', 'focus', 'focus', 'unfocus', 'unfocus',
       'unfocus', 'unfocus', 'unfocus', 'unfocus', 'unfocus', 'unfocus',
       'unfocus', 'unfocus', 'unfocus', 'focus', 'unfocus', 'unfocus',
       'unfocus', 'unfocus', 'unfocus', 'focus', 'focus', 'unfocus',
       'unfocus', 'unfocus', 'unfocus', 'unfocus', 'unfocus', 'unfocus',
       'unfocus', 'unfocus', 'unfocus', 'unfocus', 'focus', 'unfocus',
       

In [18]:
with open('D:/oprncv/test_vf/graduation_project.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [15]:
#test project
with open('D:/oprncv/test_vf/graduation_project.pkl', 'rb') as f:
    model = pickle.load(f)

In [27]:
cap = cv2.VideoCapture('D:/oprncv/UNFOCUS.mp4')
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

unfocus [0.04 0.96]
unfocus [0.07 0.93]
unfocus [0.1 0.9]
unfocus [0.29 0.71]
unfocus [0.14 0.86]
unfocus [0.01 0.99]
unfocus [0.02 0.98]
unfocus [0. 1.]
unfocus [0.02 0.98]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0.02 0.98]
unfocus [0.01 0.99]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
unfocus [0. 1.]
un